In [ ]:
import pandas as pd
import numpy as np
import polars as pl
import joblib
import random
import pathlib

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)


# Display options (optional)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

random.seed(42)
np.random.seed(42)

About the Data:
- https://meps.ahrq.gov/mepsweb/data_stats/download_data_files.jsp
- https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=1%2CHousehold+Full+Year+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Consolidated+Data


For this project, I’m working with a cleaned subset of the MEPS (Medical Expenditure Panel Survey) dataset. Specifically 2023 Full Year Consolidated Data File (PUF Number: HC251). 

MEPS is a national healthcare survey that tracks people’s demographics, insurance status, health behaviors, and medical usage and is also commonly used in fairness and healthcare modeling scenarios due to it's granularity.

In my case, I’m using MEPS because it gives me:

1. the key demographic variables I need for fairness analysis (age, sex, race/ethnicity, income, poverty level, insurance, etc.),

2. the health-status variables my model actually uses to make predictions, and enough variation across groups to see where bias shows up.

My end goal in this notebook to export all of this into a parquetdf_meta, which will become the backbone for both parts of my app:

1. Fairness Playground – where users change demographics and see how the prediction shifts.

2. Name Bias Lab – where the health profile stays constant but the first name changes.

## Goal 1

The first goal of this project is to train a predictive model that will power the Fairness Playground.

Based on the literature I reviewed — including recent work showing strong performance of boosting methods on structured healthcare data — gradient-boosting models tend to outperform more traditional approaches for this kind of problem. They capture non-linear relationships, handle mixed data types, and generally require very little feature engineering.
(Example reference: ScienceDirect, “Tabular Data Healthcare Prediction Using Gradient Boosting Methods,” 2025)
- https://www.sciencedirect.com/science/article/pii/S2588914125000140

Because of that, I’m using two state-of-the-art boosting models:

1. LightGBM
2. XGBoost

Both are fast, stable, and well-suited for tabular datasets like MEPS. They also work seamlessly with SHAP, which matters because the Fairness Playground relies on real-time model explanations whenever users change input values.

These models will form the backbone of the prediction engine used throughout the fairness module.

#### Loading and preparing the data

In [33]:
data_path = "./data/meps_2023/"
main_df = "h251.xlsx"

# reading as polars for speed
df_pl_raw = pl.read_excel(f"{data_path}{main_df}")

MEPS has hundreds of fields, but for the fairness playground I only keep the variables that matter most for understanding differences in healthcare predictions. These include core demographics (age, sex, race, ethnicity), socioeconomic indicators (income, education, poverty level), and basic access-to-care variables like insurance coverage. According to the MEPS documentation, these are the primary factors that shape both healthcare utilization and disparities across groups, so they’re essential for fairness analysis.

I also keep a small set of high-level health status and utilization variables — things like self-rated health, chronic condition flags, and counts of visits or expenditures. These are standard MEPS indicators used in most prediction studies, and they give the model enough signal to make meaningful predictions without going into overly detailed or sensitive medical fields. Keeping this narrowed, intentional subset helps the model stay interpretable and allows the fairness playground to compare predictions across groups in a controlled, transparent way.

In [ ]:
# Mapping MEPS raw variable names to cleaner names, grouped logically

rename_map = {
    # Demographics
    "DUPERSID": "person_id",
    "AGELAST": "age",
    "SEX": "sex",
    "RACEV1X": "race_simple",
    "RACETHX": "race_ethnicity",
    "HISPANX": "hispanic",
    "BORNUSA": "born_in_usa",
    "YRSINUS": "years_in_us",
    # Socioeconomic indicators
    "EDUCYR": "education_years",
    "FAMINC23": "family_income",
    "POVCAT23": "poverty_category",
    "REGION23": "region",
    # Insurance and access to care
    "INSCOV23": "insurance_coverage",
    "INSURC23": "insurance_category",
    # Self-reported health
    "RTHLTH53": "self_rated_health",
    "MNHLTH53": "self_rated_mental_health",
    "ADSMOK42": "smoker",
    # Chronic conditions
    "HIBPDX": "hypertension_dx",
    "CHDDX": "coronary_hd_dx",
    "ASTHDX": "asthma_dx",
    "DIABDX_M18": "diabetes_dx",
    # Healthcare utilization
    "OBTOTV23": "office_visits",
    "OPTOTV23": "outpatient_visits",
    "ERTOT23": "er_visits",
    "DVTOT23": "total_visits",
    "TOTEXP23": "total_expenditures",
    "IPDIS23": "inpatient_discharges",
    "IPTEXP23": "inpatient_expenditures",
    "IPNGTD23": "inpatient_nights",
}

In [ ]:
# Use the rename_map keys as keep_cols so nothing gets truncated
cols_to_keep = list(rename_map.keys())
df_pl = df_pl_raw.select(cols_to_keep).rename(rename_map)

In [36]:
# converting to pandas df
df_pd = df_pl.to_pandas()

print(df_pd.shape)

(18919, 29)


In [37]:
display(df_pd.head())

,person_id,age,sex,race_simple,race_ethnicity,hispanic,born_in_usa,years_in_us,education_years,family_income,poverty_category,region,insurance_coverage,insurance_category,self_rated_health,self_rated_mental_health,smoker,hypertension_dx,coronary_hd_dx,asthma_dx,diabetes_dx,office_visits,outpatient_visits,er_visits,total_visits,total_expenditures,inpatient_discharges,inpatient_expenditures,inpatient_nights
0,2790002101,58,2,2,3,2,1,-1,17,130700,5,2,1,1,4,3,2,2,2,2,1,3,1,0,0,646,0,0,0
1,2790002102,27,1,2,3,2,1,-1,12,130700,5,2,1,1,2,2,-1,2,2,1,2,1,0,0,2,1894,0,0,0
2,2790004101,49,2,1,2,2,1,-1,17,87000,5,2,1,1,1,1,2,2,2,2,2,1,0,0,1,986,0,0,0
3,2790006101,75,2,1,2,2,1,-1,12,38000,4,2,2,4,2,2,1,1,2,2,1,3,0,0,0,1312,0,0,0
4,2790006102,23,1,1,2,2,1,-1,11,38000,4,2,2,2,2,2,-1,2,2,2,2,0,0,0,0,0,0,0,0


### Data Cleaning

##### Handling special codes
MEPS uses special negative values such as –1 (“inapplicable”), –7 (“refused”), –8 (“don’t know”), and –9 (“not ascertained”) to indicate different types of missing data. These aren’t real values, and keeping them would distort both the summary statistics and the model training. For example, the model could mistakenly treat “–8 outpatient visits” or “–9 total expenditures” as meaningful numbers, even though they don’t represent actual behavior. To avoid that, I am going to replace these with 'NAN' before any analysis so they don't show up as fake values or categories.



#### Defining target variable
For the prediction target, I define a binary variable **`hospitalized`** based on inpatient expenditures: if a person has any positive inpatient spending, I treat that as evidence of at least one hospitalization in the year. To avoid data leakage, I drop the raw inpatient expenditure field from the features after using it to construct the label.


In [ ]:
# handling MEPS special codes for numeric columns
#    (-1, -7, -8, -9, -15 --> NaN)

SPECIAL_CODES = [-1, -7, -8, -9, -15]

numeric_cols = [
    "age",
    "education_years",
    "family_income",
    "years_in_us",
    "office_visits",
    "outpatient_visits",
    "er_visits",
    "total_visits",
    "total_expenditures",
    "inpatient_discharges",
    "inpatient_expenditures",
    "inpatient_nights",
]

df_pd[numeric_cols] = df_pd[numeric_cols].replace(SPECIAL_CODES, pd.NA)

In [ ]:
# Quick sanity check
print(df_pd[numeric_cols].describe().T)

                          count          mean           std    min      25%  \
age                     18919.0     43.716581     23.939550    0.0     23.0   
family_income           18919.0  98815.895555  91575.419638 -230.0  35000.0   
office_visits           18919.0      7.140758     13.784344    0.0      0.0   
outpatient_visits       18919.0      1.095195      4.489569    0.0      0.0   
er_visits               18919.0      0.224166      0.672914    0.0      0.0   
total_visits            18919.0      1.035943      1.688402    0.0      0.0   
total_expenditures      18919.0   8422.054125  21664.250470    0.0    299.5   
inpatient_discharges    18919.0      0.096728      0.394552    0.0      0.0   
inpatient_expenditures  18919.0   1830.599397  11540.246170    0.0      0.0   
inpatient_nights        18919.0      0.535599      4.267027    0.0      0.0   

                            50%       75%       max  
age                        45.0      64.0      85.0  
family_income         

In [ ]:
# Handling special codes for categorical columns as well
SPECIAL_CODES = [-1, -7, -8, -9, -15]

categorical_cols = [
    "sex",
    "race_simple",
    "race_ethnicity",
    "hispanic",
    "poverty_category",
    "insurance_coverage",
    "insurance_category",
    "region",
    "born_in_usa",
    "self_rated_health",
    "self_rated_mental_health",
    "smoker",
    "hypertension_dx",
    "coronary_hd_dx",
    "asthma_dx",
    "diabetes_dx",
]

# Replacing MEPS special codes with NA in categorical columns
df_pd[categorical_cols] = df_pd[categorical_cols].replace(SPECIAL_CODES, pd.NA)

In [ ]:
# Quick sanity check
print(df_pd[categorical_cols].describe().T)

                      count      mean       std  min  25%  50%  75%  max
sex                 18919.0  1.523601  0.499456  1.0  1.0  2.0  2.0  2.0
race_simple         18919.0  1.547386  1.194245  1.0  1.0  1.0  2.0  6.0
race_ethnicity      18919.0  2.148052  0.959049  1.0  2.0  2.0  2.0  5.0
hispanic            18919.0  1.778635  0.415176  1.0  2.0  2.0  2.0  2.0
poverty_category    18919.0  3.708970  1.411587  1.0  3.0  4.0  5.0  5.0
insurance_coverage  18919.0  1.485068  0.622105  1.0  1.0  1.0  2.0  3.0
insurance_category  18919.0  2.276389  1.609777  1.0  1.0  2.0  3.0  8.0


In [ ]:
# One-hot encode categorical columns
df_ohe = pd.get_dummies(df_pd, columns=categorical_cols, drop_first=False)

print(f"Shape after one-hot encoding: {df_ohe.shape}")

In [42]:
# CHeckng for missing target values
df_ohe["inpatient_expenditures"].isna().sum()

np.int64(0)

In [43]:
# Work on a copy that has valid inpatient_expenditures ()
df_model = df_ohe[df_ohe["inpatient_expenditures"].notna()].copy()

# Define target
df_model["hospitalized"] = (df_model["inpatient_expenditures"] > 0).astype(int)

# Drop leakage features from X
leakage_cols = ["inpatient_expenditures", "inpatient_nights", "inpatient_discharges"]
df_model = df_model.drop(columns=leakage_cols)

In [ ]:
"""
Used Chatgpt-5 on 21 Nov to clean and encode the feature columns
"""

'\nUsed Chatgpt-5 on 21 Nov to clean and encode the feature columns\n'

### Quick Sanity Checks and EDA

In [45]:
print(df_model.head())

    person_id  age years_in_us education_years  family_income  office_visits  \
0  2790002101   58        <NA>              17         130700              3   
1  2790002102   27        <NA>              12         130700              1   
2  2790004101   49        <NA>              17          87000              1   
3  2790006101   75        <NA>              12          38000              3   
4  2790006102   23        <NA>              11          38000              0   

   outpatient_visits  er_visits  total_visits  total_expenditures  sex_1  \
0                  1          0             0                 646  False   
1                  0          0             2                1894   True   
2                  0          0             1                 986  False   
3                  0          0             0                1312  False   
4                  0          0             0                   0   True   

   sex_2  race_simple_1  race_simple_2  race_simple_3  race_si

In [46]:
# Checking for class imbalance
print(df_model["hospitalized"].value_counts(normalize=True))

hospitalized
0    0.926265
1    0.073735
Name: proportion, dtype: float64


Only 7.37% of individuals in the MEPS dataset experienced any inpatient hospitalization while 93.7% did not, which means the dataset is extremely imbalanced. Nonetheless this makes because hospitalization is relatively rare

In [47]:
# Basic info
print(df_model.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18919 entries, 0 to 18918
Data columns (total 67 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   person_id                   18919 non-null  object
 1   age                         18919 non-null  int64 
 2   years_in_us                 3080 non-null   object
 3   education_years             17713 non-null  object
 4   family_income               18919 non-null  int64 
 5   office_visits               18919 non-null  int64 
 6   outpatient_visits           18919 non-null  int64 
 7   er_visits                   18919 non-null  int64 
 8   total_visits                18919 non-null  int64 
 9   total_expenditures          18919 non-null  int64 
 10  sex_1                       18919 non-null  bool  
 11  sex_2                       18919 non-null  bool  
 12  race_simple_1               18919 non-null  bool  
 13  race_simple_2               18919 non-null  bo

Overall, this processed dataset currently contains 18,919 rows and 94 columns. Most of the features are one-hot encoded categorical variables stored as booleans (great for boosting methods), along with a set of numeric fields capturing age, income, visits, and expenditures. Both, `years_in_us` and `education_years` still come in as object types because MEPS encodes missing data using negative codes. I’ll convert both to numeric and impute their missing values using the median so the model receives clean, consistent inputs.

Aside from those two fields, the dataset has no remaining missing values, and all categorical variables have already been fully expanded into dummy variables. The wide structure means the model can learn group-specific patterns while still being stable for gradient boosting. Overall, once those two numeric fields are fixed and imputed, the dataset is fully ready for supervised learning.

In [48]:
# Converting columns to numeric
df_model["years_in_us"] = pd.to_numeric(df_model["years_in_us"], errors="coerce")
df_model["education_years"] = pd.to_numeric(
    df_model["education_years"], errors="coerce"
)

# Imputing columns
df_model["years_in_us"] = df_model["years_in_us"].fillna(
    df_model["years_in_us"].median()
)
df_model["education_years"] = df_model["education_years"].fillna(
    df_model["education_years"].median()
)

### Splitting datasets

To evaluate the model properly, I will split the dataset into training and testing sets using a stratified split. Since hospitalization is relatively rare in this MEPS sample (with around 7% of cases), stratification would ensure that both subsets maintain the same class balance. This prevents the model from being trained or evaluated on an artificially skewed distribution and will give a more realistic sense of performance.

In addition, because of the imbalance, I will use StratifiedKFold for cross-validation wherein each fold will contain roughly the same proportion of hospitalized vs. non-hospitalized cases as the full dataset. That consistency avoids folds where the minority class is underrepresented or missing entirely, which can lead to unstable or overly optimistic estimates. Using stratified folds makes the evaluation much more reliable and gives a clearer picture of how well the model will generalize.

In [49]:
# Keep person_id separately so we can split it alongside X and y
ids = df_model["person_id"]

# Features and target
X = df_model.drop(columns=["hospitalized"])
y = df_model["hospitalized"]

# Stratified train/test split to respect imbalance
X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
    X,
    y,
    ids,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

X_train.shape, X_test.shape, y_train.value_counts(normalize=True), y_test.value_counts(
    normalize=True
)

# Now we drop Person_ids from X_train, X_test
X_train = X_train.drop(columns=["person_id"])
X_test = X_test.drop(columns=["person_id"])

In [50]:
# Stratified 5-fold cross-validation to preserve class balance in each fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### Modeling

#### LightGBM

In [51]:
# LightGBM model (tuned lightly for tabular + imbalance)
lgb_model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.03,
    class_weight="balanced",
    random_state=42,
)

lgb_cv_scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), start=1):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    lgb_model.fit(X_tr, y_tr)
    val_pred = lgb_model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_pred)
    lgb_cv_scores.append(auc)
    print(f"Fold {fold} ROC-AUC: {auc:.3f}")

print(f"\nMean ROC-AUC: {np.mean(lgb_cv_scores):.3f} ± {np.std(lgb_cv_scores):.3f}")

[LightGBM] [Info] Number of positive: 893, number of negative: 11215
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002758 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 897
[LightGBM] [Info] Number of data points in the train set: 12108, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Fold 1 ROC-AUC: 0.958
[LightGBM] [Info] Number of positive: 893, number of negative: 11215
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001426 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 894
[LightGBM] [Info] Number of data points in the train set: 12108, number of used features: 64
[L

#### XGBoost

In [52]:
# Handle class imbalance for XGBoost
pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

xgb_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.03,
    eval_metric="logloss",
    scale_pos_weight=pos_weight,  # imbalance handling
    random_state=42,
)

xgb_cv_scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), start=1):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    xgb_model.fit(X_tr, y_tr)
    val_pred = xgb_model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_pred)
    xgb_cv_scores.append(auc)
    print(f"Fold {fold} ROC-AUC: {auc:.3f}")

print(f"\nMean ROC-AUC: {np.mean(xgb_cv_scores):.3f} ± {np.std(xgb_cv_scores):.3f}")

Fold 1 ROC-AUC: 0.955
Fold 2 ROC-AUC: 0.958
Fold 3 ROC-AUC: 0.957
Fold 4 ROC-AUC: 0.956
Fold 5 ROC-AUC: 0.957

Mean ROC-AUC: 0.956 ± 0.001


After running stratified 5-fold cross-validation on both LightGBM and XGBoost, the two models performed almost identically. Each model had a mean ROC-AUC of roughly 0.956-0.957 with very small standard deviations across folds. This means both gradient-boosting methods are strong and stable for this datasetwith essentially negligile differences.

Despite the similar results, I’m choosing LightGBM as the final model. It trains faster, has lower memory overhead, and integrates more smoothly with SHAP for real-time explainability in the Fairness Playground. Since responsiveness and interpretability are core goals of the app, LightGBM offers a cleaner user experience without sacrificing predictive performance.

### Train the final LightGBM model on the full training set

In [53]:
# Final LightGBM model (same settings as CV)
final_lgb = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.03,
    class_weight="balanced",
    random_state=42,
)

# Train on full training data
final_lgb.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 1116, number of negative: 14019
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002165 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 907
[LightGBM] [Info] Number of data points in the train set: 15135, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


LGBMClassifier(class_weight='balanced', learning_rate=0.03, n_estimators=500,
               random_state=42)

### Evaluate on Test Set

In [54]:
# Predict
y_proba_test = final_lgb.predict_proba(X_test)[:, 1]
y_pred_test = final_lgb.predict(X_test)

# Metrics
print(f"ROC-AUC: {roc_auc_score(y_test, y_proba_test):.3f}")
print(f"Precision: {precision_score(y_test, y_pred_test):.3f}")
print(f"Recall: {recall_score(y_test, y_pred_test):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred_test):.3f}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred_test))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))

ROC-AUC: 0.958
Precision: 0.485
Recall: 0.785
F1 Score: 0.599

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.93      0.96      3505
           1       0.48      0.78      0.60       279

    accuracy                           0.92      3784
   macro avg       0.73      0.86      0.78      3784
weighted avg       0.95      0.92      0.93      3784

Confusion Matrix:
[[3272  233]
 [  60  219]]


Interpreting the Model’s Test Performance

The final LightGBM model performs well on the test set. The ROC-AUC is 0.958, which means the model is very good at separating hospitalized vs. non-hospitalized individuals. The model also catches most of the people who actually were hospitalized with a recall for the positive class as 0.785. Precision is lower at 0.485, which is expected given how rare hospitalization is, but the F1-score (0.599) shows the model is balancing things reasonably well.

Looking at the confusion matrix, the model correctly identifies 219 out of 279 hospitalized cases, while keeping false positives at a manageable level. Overall accuracy is about 92%, which lines up with the rest of the metrics.


Overall, this level of performance is good enough for moving into explainability and building my fairness playground. A weak or noisy model would have made it hard to trust any fairness patterns, since differences across demographic groups would have just come from randomness. However here, the model is clearly learning real signals from the data, and it generalizes well.

That means when I start showing SHAP explanations or let users tweak demographic attributes in the Fairness Playground, the changes in predictions will be meaningful.

### Storing everythign

For fairness and explainability work, I also need the original demographic variables (age, sex, race/ethnicity, poverty level, insurance coverage, etc.). These are essential for checking whether the model behaves differently across groups, but they don’t belong inside the modeling matrix after one-hot encoding and leakage removal.

So here, I extract a clean subset of the dataset containing only the human-readable demographic attributes plus person_id, and save it as df_meta. This makes it much easier in the fairness notebook to merge predictions back with the correct demographic information, without trying to reconstruct categories from dummy variables.

In [55]:
# SAVE df_meta (demographic attributes only)

meta_cols = [
    "person_id",
    "age",
    "sex",
    "race_ethnicity",
    "hispanic",
    "poverty_category",
    "insurance_coverage",
    "family_income",
    "self_rated_health",
    "self_rated_mental_health",
]

df_meta = df_pd[meta_cols].copy()

display(df_meta.head())

,person_id,age,sex,race_ethnicity,hispanic,poverty_category,insurance_coverage,family_income,self_rated_health,self_rated_mental_health
0,2790002101,58,2,3,2,5,1,130700,4,3
1,2790002102,27,1,3,2,5,1,130700,2,2
2,2790004101,49,2,2,2,5,1,87000,1,1
3,2790006101,75,2,2,2,4,2,38000,2,2
4,2790006102,23,1,2,2,4,2,38000,2,2


In short df_meta will help me keep the original sensitive attributes separate from the modeling data, making fairness analysis simpler, safer, and more transparent.

In [56]:
# Creating a folder to store artifacts
artifacts_dir = pathlib.Path("fairness_artifacts")
artifacts_dir.mkdir(exist_ok=True)

In [57]:
# Saving the attribute table
df_meta.to_parquet(artifacts_dir / "df_meta.parquet", index=False)
ids_train.to_frame(name="person_id").to_parquet(
    artifacts_dir / "ids_train.parquet", index=False
)
ids_test.to_frame(name="person_id").to_parquet(
    artifacts_dir / "ids_test.parquet", index=False
)

In [58]:
# Saving train/test feature matrices as Parquet
X_train_path = artifacts_dir / "X_train.parquet"
X_test_path = artifacts_dir / "X_test.parquet"

X_train.to_parquet(X_train_path, index=False)
X_test.to_parquet(X_test_path, index=False)

In [59]:
# Saving train/test targets as Parquet
# also converting train/test targets to df to ensure clean schema
y_train_path = artifacts_dir / "y_train.parquet"
y_test_path = artifacts_dir / "y_test.parquet"

y_train.to_frame(name="hospitalized").to_parquet(y_train_path, index=False)
y_test.to_frame(name="hospitalized").to_parquet(y_test_path, index=False)

In [ ]:
# Saving the trained LightGBM model
model_path = artifacts_dir / "final_lightgbm_model.pkl"
joblib.dump(final_lgb, model_path)

print("Saved:")
print(f"X_train:{X_train_path}")
print(f"X_test: {X_test_path}")
print(f"y_train: {y_train_path}")
print(f"y_test: {y_test_path}")
print(f"model: {model_path}")

Saved:
X_train:fairness_artifacts/X_train.parquet
X_test: fairness_artifacts/X_test.parquet
y_train: fairness_artifacts/y_train.parquet
y_test: fairness_artifacts/y_test.parquet
model: fairness_artifacts/final_lightgbm_model.pkl
